In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import os 
import pickle
import sys 
import time
import tqdm
import pandas as pd

from lfimodels.channelomics.ChannelSingle import ChannelSingle
from lfimodels.channelomics.ChannelStats import ChannelStats
%matplotlib inline

In [ ]:
mpl_params = {'legend.fontsize': 18,
              'legend.frameon': False,
                      'axes.titlesize': 18,
                      'axes.labelsize': 16,
                      'xtick.labelsize': 14,
                      'ytick.labelsize': 14,
             'figure.figsize' : (18, 5)}

mpl.rcParams.update(mpl_params)

## Voltage-clamp protocol

In [ ]:
prot = ['v_act','v_inact','v_deact','v_ap','v_ramp']
p = 0
protocols = \
pd.read_csv('~/Dropbox/Master/mackelab/code/lfi-models/lfimodels/channelomics/protocols/k_channels/'+prot[p]+'.dat',sep='\t')

protocols = protocols.drop(protocols.columns[-1],axis=1)

# extract voltages and time
protocols1 = protocols.values

V = protocols1[:,1:].T
t = protocols1[:,0]   # attention: time step varies, but we will assume that it is constant
tstep = np.mean(np.diff(t)) # ms

time_stamp = time.strftime('%Y%m%d%H%M_')

In [ ]:
Vs = {}
ts = {}

for i, p in enumerate(prot): 
    protocols = pd.read_csv('~/Dropbox/Master/mackelab/code/lfi-models/lfimodels/channelomics/protocols/k_channels/'+prot[i]+'.dat',sep='\t')
    protocols = protocols.drop(protocols.columns[-1],axis=1)

    # extract voltages and time
    protocols1 = protocols.values

    Vs[p] = protocols1[:,1:].T
    ts[p] = protocols1[:,0]   # attention: time step varies, but we will assume that it is constant
    

In [ ]:
channel_type = 'kslow'

In [ ]:
GT = {'kd': np.array([4, -63, 0.032, 15, 5, 0.5, 10, 40]),
      'kslow': np.array([1, 35, 10, 3.3, 20])}
LP = {'kd': [r'$M$', r'$V_T$', r'$R_{\alpha}$', r'$th_{\alpha}$', r'$q_{\alpha}$', r'$R_{\beta}$', r'$th_{\beta}$', r'$q_{\beta}$'],
      'kslow': [r'$M$', r'$th_{p}$', r'$q_{p}$', r'$R_{\tau}$', r'$q_{\tau}$']}

E_channel = {'kd': -90.0, 'kslow': -90.0}
fact_inward = {'kd': 1, 'kslow': 1}

cython = True

gt = GT[channel_type]
n_params = len(gt)
labels_params = LP[channel_type]
prior_lims = np.sort(np.concatenate((0.3 * gt.reshape(-1, 1), 1.3 * gt.reshape(-1, 1)), axis=1))

In [ ]:
# generate observed data
n_params_obs = len(gt)
m_obs = ChannelSingle(channel_type=channel_type, n_params=n_params_obs, cython=cython)
s = ChannelStats(channel_type=channel_type)

xo = m_obs.gen(gt.reshape(1,-1))
xo_stats = s.calc(xo[0])

In [ ]:
fig, ax = plt.subplots(3, 5, figsize=(18, 9), sharey='row', sharex='col', gridspec_kw = {'height_ratios':[1, 2, 2]})
prot = ['v_act','v_inact','v_deact','v_ap','v_ramp']
prot_labels = ['acitvation', 'inactivation', 'deactivation', 'action potentials', 'ramping']
channel_types = ['kd', 'kslow']

for ii, (k, v) in enumerate(Vs.items()): 
    
    # set colors 
    num_levels = v.shape[0]
    cm1 = mpl.cm.viridis
    col1 = [cm1(1. * k / num_levels) for k in range(num_levels)]
    
    ax[0, ii].set_title(prot_labels[ii])
    ax[2, ii].set_xlabel('time [ms]')
    
    plt.sca(ax[0, ii])
    for jj, subv in enumerate(v):
        plt.plot(ts[k], subv, color='r')
    
    
    for ci, ch in enumerate(channel_types): 
        n_params_obs = len(gt)
        m_obs = ChannelSingle(channel_type=ch, n_params=n_params_obs, cython=cython)
        s = ChannelStats(channel_type=ch)
        gt = GT[ch]
        
        xo = m_obs.gen(gt.reshape(1,-1))
        xo_stats = s.calc(xo[0])
        I = xo[0][0][prot[ii]]['data']
        t = xo[0][0][prot[ii]]['time']    

        plt.sca(ax[ci + 1, ii])
        for j in range(num_levels):
            plt.plot(t, xo[0][0][prot[ii]]['data'][j,], color = col1[j], lw=2)
    
ax[0, 0].set_ylabel('voltage [mV]')
ax[1, 0].set_ylabel('norm. current')
ax[2, 0].set_ylabel('norm. current')
plt.tight_layout()

In [ ]:
fn = time_stamp + 'voltageprots_currentraces_bothchannels.png'.format(channel_type)
fig.savefig(os.path.join('../figures', fn), dpi=300)